<a href="https://colab.research.google.com/github/AymenMetallaoui/AymenMetallaoui/blob/main/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout

import itertools

import torch

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import os
import numpy as np
import pandas as pd
import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
#Second change your current folder to point working folder My Drive
os.chdir("/content/drive/My Drive/")
os.listdir()

['Operating System Concepts (9th Ed) - Gagne, Silberschatz, and Galvin.pdf',
 'school',
 'screens',
 'crib sheet COMP478.gdoc',
 'Colab Notebooks',
 'dataset',
 'Untitled document.gdoc']

In [7]:
# Use GPU when possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#directory dataset
DATASET_DIR ='./dataset/dataset'




cpu


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/dataset/train',
    target_size=(512, 512), # modify target size to 512x512
    batch_size=4,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/dataset/train',
    target_size=(512, 512), # modify target size to 512x512
    batch_size=4,
    class_mode='binary',
    subset='validation'
)


Found 79 images belonging to 2 classes.
Found 19 images belonging to 2 classes.


In [ ]:


# set the directory of the training data
train_dir = '/content/drive/MyDrive/dataset/dataset/train'

# list the class names
classes = ['covid', 'normal']

# loop over each class and display the first two images
for class_name in classes:
    class_path = os.path.join(train_dir, class_name)
    image_names = os.listdir(class_path)[:2]  # get first two images in the folder

    for image_name in image_names:
        image_path = os.path.join(class_path, image_name)
        image = Image.open(image_path)
        image.show(title=class_name)


In [10]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=4
)

# plot training/validation loss and accuracy
epochs_range = range(1, 5)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()


In [ ]:
# predict labels of validation set
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)

# calculate and print confusion matrix
cm = confusion_matrix(validation_generator.classes, y_pred)
print(cm)

# plot confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# calculate accuracy, precision, recall, and F-score
tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f_score = 2 * (precision * recall) / (precision + recall)

# print accuracy, precision, recall, and F-score
print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision: {:.2f}%".format(precision * 100))
print("Recall: {:.2f}%".format(recall * 100))
print("F-score: {:.2f}%".format(f_score * 100))

In [ ]:
# load pre-trained VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# freeze all the layers in the pre-trained model
for layer in vgg16_model.layers:
    layer.trainable = False

# add custom classification layers on top of the pre-trained model
model = Sequential()
model.add(vgg16_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# print model summary
print(model.summary())


In [ ]:
# train the model
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

# plot the training and validation accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# predict labels of validation set
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)

# calculate and print confusion matrix
cm = confusion_matrix(validation_generator.classes, y_pred)
print(cm)

# calculate accuracy, F-score, recall, and precision
accuracy = accuracy_score(validation_generator.classes, y_pred)
fscore = f1_score(validation_generator.classes, y_pred)
recall = recall_score(validation_generator.classes, y_pred)
precision = precision_score(validation_generator.classes, y_pred)

print('Accuracy:', accuracy)
print('F-score:', fscore)
print('Recall:', recall)
print('Precision:', precision)

In [ ]:
# plot confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# create the model
base_model = InceptionV3(input_shape=(512, 512, 3), include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=x)

# freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# train the model
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

# plot the training and validation accuracy and loss
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# predict labels of validation set
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)

# calculate and print confusion matrix
cm = confusion_matrix(validation_generator.classes, y_pred)
print(cm)

# calculate accuracy, F-score, recall, and precision
accuracy = accuracy_score(validation_generator.classes, y_pred)
fscore = f1_score(validation_generator.classes, y_pred)
recall = recall_score(validation_generator.classes, y_pred)
precision = precision_score(validation_generator.classes, y_pred)

print('Accuracy:', accuracy)
print('F-score:', fscore)
print('Recall:', recall)
print('Precision:', precision)


In [ ]:
# plot confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()